Alan Patricio González Bernal - A01067546
Entrega Mid-Term de modelo Wa-Tor


Introducción
Wa-Tor es una simulación de peces contra tiburones en un mundo toroidal. Fue diseñado por AK Dewdney en 1984 y publicado en Scientific American bajo el nombre “Tiburones y peces libran una guerra ecológica en el planeta toroidal Wa-Tor”.

Tu reto será repetir esta simulación. Así de sencillo.

Descripción
El mundo de Wa-Tor consiste en una cuadrícula bidimensional de celdas que pueden estar vacías, contener un pez o contener un tiburón. La cuadrícula se envuelve de arriba a abajo y de izquierda a derecha, por lo que también se puede considerar como un toroide (de ahí Wa-Tor).

El tiempo avanza en pasos discretos, llamados cronones. En cada cronón, el estado de cada criatura evoluciona de acuerdo con las siguientes reglas:

Todas las criaturas empiezan en una celda vacía;
Cada pez se mueve aleatoriamente a una celda adyacente (usando un vecindario de von Neumann de celdas vecinas directamente al norte, sur, este y oeste); si las celdas están todas ocupadas, no se mueve;
Después de que un pez ha sobrevivido un número fijo de cronones (fertility_threshold), se reproduce dejando otro pez atrás en su celda anterior después de moverse. Posteriormente, su fertilidad se pone a cero;
Después de cada cronón, cada pez pierde una unidad de energía. Si la energía de un pez llega a cero, muere;
Cada tiburón se mueve aleatoriamente a una celda adyacente (usando un vecindario de von Neumann de celdas vecinas directamente al norte, sur, este y oeste) ocupada por un pez y gana una cierta cantidad de energía al hacerlo (al "comerse" el pez); si las celdas adyacentes están todas vacías, se mueve a una de ellas al azar; si las celdas adyacentes están llenas de tiburones, no se mueve;
Después de cada cronón, cada tiburón pierde una unidad de energía. Si la energía de un tiburón llega a cero, muere;
Una vez que un tiburón alcanza su propio umbral de fertilidad, se reproduce de la misma manera que los peces.
Parámetros de la simulación
Wa-Tor está controlado por los siguientes parámetros:

Tamaño del mundo: 75 x 50.
Cantidad de peces: 120
Cantidad de tiburones: 40
Energía inicial de los peces: 20
Energía inicial de los tiburones: 3
Umbral de fertilidad de los peces (fertility_threshold): 4
Umbral de fertilidad de los tiburones (fertility_threshold): 12

In [128]:
# Se generan los imports, los mismos que el segregation model
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''RandomActivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime


In [ ]:
# Voy a tratar de rehacer el agente en este bloque, quiero poder comprenderlo al 100% y rectificar errores, pero no borro el otro agente para no perderlo.

In [129]:
# Se define el agente
# Necesito que si es tiburón sea de color rojo, si es pez sea de color azul
# Si es tiburón, que se mueva a una celda vacía, si es pez que se mueva a una celda vacía o a una celda con pez
# Si es tiburón, que se coma al pez, si es pez que no haga nada
# Si es tiburón, que se reproduzca con otro tiburón, si es pez que se reproduzca con otro pez
# Si es tiburón, que muera si no se ha comido un pez en 3 pasos, si es pez que muera si no se ha reproducido en 3 pasos

class WaTorAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.live = np.random.choice([0, 1])
        self.next_state = None
    
    def step(self):
        neighbors = self.model.grid.get_neighbors(self.pos,
                                                moore = True,
                                                include_center = False)
        if self.live == 1:
            self.move(neighbors)
            self.eat(neighbors)
            self.reproduce(neighbors)
            self.die()
    
    def move(self, neighbors):
        if self.next_state == None:
            empty_cells = [cell for cell in neighbors if self.model.grid.is_cell_empty(cell)]
            if empty_cells:
                new_position = self.random.choice(empty_cells)
                self.model.grid.move_agent(self, new_position)
    
    def eat(self, neighbors):
        if self.next_state == None:
            fish_cells = [cell for cell in neighbors if self.model.grid.is_cell_empty(cell) == False]
            if fish_cells:
                fish = self.random.choice(fish_cells)
                self.model.grid.remove_agent(fish)
                self.next_state = 1
    
    def reproduce(self, neighbors):
        if self.next_state == None:
            empty_cells = [cell for cell in neighbors if self.model.grid.is_cell_empty(cell)]
            if empty_cells:
                new_position = self.random.choice(empty_cells)
                self.model.grid.place_agent(WaTorAgent(self.model.next_id(), self.model), new_position)
    
    def die(self):
        if self.next_state == None:
            self.live = 0
        else:
            self.live = self.next_state
        self.next_state = None
    
    def advance(self):
        pass
    

In [130]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for (content, (x, y)) in model.grid.coord_iter():
        if content is not None:
            grid[x][y] = content.live
    return grid
